In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openpyxl


df=pd.read_excel('patents_data_999p.xlsx')


In [2]:
df.head()

,status,title,ap_num,application_date,applicant,representative,final_owner,priority_date,international_filing_date,summary
0,등록,자동화 폐기물 수거함의 딥러닝 기반 인공지능 운영 시스템 및 방법(OPERATION...,1020200078616,20200626,(주)에이씨아이케미칼아시아,고영갑,(주)에이씨아이케미칼아시아,NaN,NaN,"본 발명의 실시 예에 따른 시스템은, 자동화 폐기물 수거함의 딥러닝 기반 인공지능 ..."
1,등록,"인공지능(AI)을 통한 딥러닝훈련모듈과, 순위화프레임워크모듈을 활용하여, 법률전문가...",1020170099431,20170807,강준철,이영규,강소라,NaN,NaN,"본 발명은, 인공지능(AI)에 관한 기술과 온라인 문자열 사전에 관한 기술을 활용하..."
2,공개,인공지능을 이용한 품질검사 시스템 및 방법(System and method for ...,1020220014151,20220203,모던에이아이비전솔루션 주식회사,김정수,NaN,NaN,NaN,"본 발명은 품질검사를 위한 장치 및 방법에 관한 것으로, 본 발명에 따르면, 종래,..."
3,공개,파이썬 딥러닝 기반 인공지능(AI) 부동산 컨설팅 매물장(Python deep le...,1020210106993,20210812,조국환,NaN,NaN,NaN,NaN,본 발명은 광범위하게 활용되는 인터넷기반 부동산 정보를 때와 장소를 불문하고 기록하...
4,거절,PI 기반 빅데이터 퍼스널 컬러 진단 인공지능 딥러닝 알고리즘 시스템 운영방법(PI...,1020230034713,20230316,주식회사 오콜로르,박상열,NaN,NaN,NaN,본 발명은 PI(Personal Identity) 정보 빅데이터를 구축하는 통하여 ...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89910 entries, 0 to 89909
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   status                     89910 non-null  object 
 1   title                      89910 non-null  object 
 2   ap_num                     89910 non-null  int64  
 3   application_date           89910 non-null  int64  
 4   applicant                  89910 non-null  object 
 5   representative             87328 non-null  object 
 6   final_owner                39703 non-null  object 
 7   priority_date              25978 non-null  float64
 8   international_filing_date  13704 non-null  float64
 9   summary                    89908 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 6.9+ MB


In [ ]:
categories = {
    '제너럴': r'\b(추론|알고리즘|학습|훈련방법|지도방법|학습방법)\b',
    '의료': r'\b(의료|헬스|치료|건강|유전자|병원|환자)\b',
    '전자상거래': r'\b(상거래|상품|배송|판매|결제|공급망|판매망)\b',
    '자동차': r'\b(자동차|주행|자율주행|도로|차량|교통|보행자|차선|신호등)\b',
    '금융': r'\b(금융|코인|트레이딩|투자|신용|금융사기|자산)\b',
    '교육': r'\b(교육|강의|교육용|시험|성적|학생)\b',
    '농업': r'\b(농업|스마트농업|농작물|작물|농약|수확|스마트팜)\b',
    '엔터테인먼트': r'\b(콘텐츠생성|게임|미디어|사용자인터페이스|가상현실|음성인식)\b',
    '보안': r'\b(침입|탐지|데이터보호|위협예측|인증시스템|보안|암호화)\b'
}

In [14]:
!pip install konlpy

   ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
   ---------------------------------------- 0.2/19.4 MB 3.1 MB/s eta 0:00:07
    --------------------------------------- 0.3/19.4 MB 3.3 MB/s eta 0:00:06
   - -------------------------------------- 0.5/19.4 MB 3.5 MB/s eta 0:00:06
   - -------------------------------------- 0.7/19.4 MB 3.4 MB/s eta 0:00:06
   - -------------------------------------- 0.8/19.4 MB 3.5 MB/s eta 0:00:06
   -- ------------------------------------- 1.0/19.4 MB 3.6 MB/s eta 0:00:06
   -- ------------------------------------- 1.2/19.4 MB 3.9 MB/s eta 0:00:05
   -- ------------------------------------- 1.4/19.4 MB 3.9 MB/s eta 0:00:05
   --- ------------------------------------ 1.6/19.4 MB 4.0 MB/s eta 0:00:05
   --- ------------------------------------ 1.8/19.4 MB 4.0 MB/s eta 0:00:05
   ---- ----------------------------------- 2.0/19.4 MB 4.1 MB/s eta 0:00:05
   ---- ----------------------------------- 2.2/19.4 MB 4.0 MB/s eta 0:00:05
   ---

In [6]:
from konlpy.tag import Okt
import pandas as pd
import re
from collections import Counter

# KoNLPy 형태소 분석기 초기화
okt = Okt()

# 전체 텍스트 합치기
df['title'] = df['title'].astype(str)
df['summary'] = df['summary'].astype(str)

# 텍스트 전처리 및 토큰화 함수 정의
def preprocess_and_tokenize(text):
    text = re.sub(r'[^ ㄱ-ㅣ가-힣]+', '', text)  # 한글만 남기고 모든 문자 제거
    nouns = okt.nouns(text)  # 형태소 분석 후 명사 추출
    nouns = [word for word in nouns if len(word) > 1]  # 한 글자 이상의 명사만 추출
    return nouns

# 각 행별로 키워드 추출 및 누적
all_keywords = []
for index, row in df.iterrows():
    text = row['title'] + ' ' + row['summary']
    tokens = preprocess_and_tokenize(text)
    all_keywords.extend(tokens)

# 전체 텍스트에서 빈도 계산 및 상위 빈출 키워드 추출
word_freq = Counter(all_keywords)
top_keywords = word_freq.most_common(500)

# 결과 출력
print(top_keywords)

[('상기', 399645), ('장치', 146819), ('방법', 138421), ('포함', 136482), ('정보', 134298), ('데이터', 109246), ('단계', 96790), ('시스템', 82106), ('제공', 68079), ('사용자', 66454), ('발명', 65683), ('하나', 59760), ('기반', 59046), ('이용', 57886), ('이미지', 49925), ('영상', 47418), ('대한', 46509), ('전자', 46104), ('제어', 45868), ('복수', 44691), ('학습', 42495), ('입력', 39610), ('실시', 39441), ('모델', 38256), ('수신', 36930), ('통해', 34737), ('인공', 34282), ('기초', 34013), ('지능', 33316), ('신호', 32993), ('로부터', 32389), ('수행', 31946), ('획득', 31804), ('분석', 31678), ('개시', 31253), ('모듈', 30617), ('서버', 29934), ('처리', 29491), ('영역', 28680), ('저장', 28013), ('통신', 27967), ('동작', 27253), ('위치', 27188), ('서비스', 26419), ('차량', 26134), ('관리', 26019), ('이상', 25884), ('센서', 25318), ('출력', 24077), ('상태', 23736), ('결정', 23678), ('프로세서', 23602), ('예측', 23506), ('구성', 23164), ('대응', 23122), ('전송', 22232), ('설정', 22178), ('단말', 21991), ('특징', 21826), ('연결', 21614), ('디스플레이', 21496), ('객체', 19792), ('인식', 19523), ('결과', 18795), ('추출', 18378), ('각각', 

In [12]:
import re

# 각 카테고리와 해당 정규표현식 패턴 정의
categories = {
    '제너럴': r'\b(추론|알고리즘|학습|훈련방법|지도방법|학습방법|머신러닝|패턴인식)\b',
    '의료': r'\b(의료|헬스|치료|건강|유전자|병원|환자|의료기기|의료정보|바이오의료)\b',
    '전자상거래서비스': r'\b(상거래|상품|배송|판매|결제|공급망|판매망|서비스|전자상거래플랫폼|온라인쇼핑|디지털결제)\b',
    '자동차': r'\b(자동차|주행|자율주행|도로|차량|교통|보행자|차선|신호등)\b',
    '금융': r'\b(금융|코인|트레이딩|투자|신용|금융사기|자산|블록체인|디지털자산)\b',
    '교육': r'\b(교육|강의|교육용|시험|성적|학생|학습관리)\b',
    '농업': r'\b(농업|스마트농업|농작물|작물|농약|수확|스마트팜)\b',
    '엔터테인먼트': r'\b(콘텐츠생성|게임|미디어|사용자인터페이스|가상현실|음성인식|가상현실콘텐츠|엔터테인먼트)\b',
    '보안': r'\b(보안|침입|탐지|데이터보호|위협예측|인증시스템|보안|암호화|안전기술)\b',
    '자동화시스템': r'\b(자동화|로봇자동화|제어시스템|IoT|위협예측|인증시스템|보안|암호화|안전기술|로봇제어|스마트시스템)\b'
}

# # check_keywords 함수 정의
# def check_keywords(text, pattern):
#     return 1 if re.search(pattern, text, flags=re.IGNORECASE) else 0

# 공백을 제거하고 비교하는 함수 정의
def check_keywords(text, pattern):
    # 문자열의 공백 제거
    text = text.replace(" ", "")
    return 1 if re.search(pattern, text, flags=re.IGNORECASE) else 0


# 결측치 처리 후 각 카테고리에 대해 title이나 summary에서 키워드를 찾아서 하나의 컬럼에 표시
for category, pattern in categories.items():
    df[category] = df.apply(lambda row: check_keywords((row['title'] if isinstance(row['title'], str) else '') or (row['summary'] if isinstance(row['summary'], str) else ''), pattern), axis=1)


df


,status,title,ap_num,application_date,applicant,representative,final_owner,priority_date,international_filing_date,summary,제너럴,의료,전자상거래,자동차,금융,교육,농업,엔터테인먼트,보안
0,등록,자동화 폐기물 수거함의 딥러닝 기반 인공지능 운영 시스템 및 방법(OPERATION...,1020200078616,20200626,(주)에이씨아이케미칼아시아,고영갑,(주)에이씨아이케미칼아시아,NaN,NaN,"본 발명의 실시 예에 따른 시스템은, 자동화 폐기물 수거함의 딥러닝 기반 인공지능 ...",0,0,0,0,0,0,0,0,0
1,등록,"인공지능(AI)을 통한 딥러닝훈련모듈과, 순위화프레임워크모듈을 활용하여, 법률전문가...",1020170099431,20170807,강준철,이영규,강소라,NaN,NaN,"본 발명은, 인공지능(AI)에 관한 기술과 온라인 문자열 사전에 관한 기술을 활용하...",0,0,0,0,0,0,0,0,0
2,공개,인공지능을 이용한 품질검사 시스템 및 방법(System and method for ...,1020220014151,20220203,모던에이아이비전솔루션 주식회사,김정수,NaN,NaN,NaN,"본 발명은 품질검사를 위한 장치 및 방법에 관한 것으로, 본 발명에 따르면, 종래,...",0,0,0,0,0,0,0,0,0
3,공개,파이썬 딥러닝 기반 인공지능(AI) 부동산 컨설팅 매물장(Python deep le...,1020210106993,20210812,조국환,NaN,NaN,NaN,NaN,본 발명은 광범위하게 활용되는 인터넷기반 부동산 정보를 때와 장소를 불문하고 기록하...,0,0,0,0,0,0,0,0,0
4,거절,PI 기반 빅데이터 퍼스널 컬러 진단 인공지능 딥러닝 알고리즘 시스템 운영방법(PI...,1020230034713,20230316,주식회사 오콜로르,박상열,NaN,NaN,NaN,본 발명은 PI(Personal Identity) 정보 빅데이터를 구축하는 통하여 ...,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89905,공개,셀 전압 검증을 통한 배터리 제어(BATTERY CONTROL WITH CELL V...,1020247013462,20220922,볼보 카 코오포레이션,양영준,NaN,20210923.0,20220922.0,"본 개시는 전기 및/또는 하이브리드 전기 차량 구동 기술에 관한 것이다. 특히, 전...",0,0,0,0,0,0,0,0,0
89906,공개,인접 평면 교란을 회피하기 위해 문제있는 메모리 평면의 격리(ISOLATING PR...,1020237040022,20220503,"웨스턴 디지털 테크놀로지스, 인코포레이티드",양영준,NaN,20210722.0,20220503.0,프로그램 또는 소거와 같은 다중 평면 동작에서 메모리 셀들의 결함 블록들을 검출하고...,0,0,0,0,0,0,0,0,0
89907,공개,비디오 코딩을 위한 신경망의 하이브리드 훈련 방법 및 장치(METHODS AND A...,1020237032052,20220302,베이징 다지아 인터넷 인포메이션 테크놀로지 컴퍼니 리미티드,유미특허법인,NaN,20210303.0,20220302.0,비디오 코딩을 위한 신경망의 하이브리드 훈련 방법 및 장치가 제공된다. 이 방법은 ...,0,0,0,0,0,0,0,0,0
89908,공개,항균 펩타이드 및 이의 사용 방법(ANTIMICROBIAL PEPTIDES AND ...,1020237023202,20181205,조에티스 서비시즈 엘엘씨,제일특허법인(유),NaN,20171207.0,20181205.0,일반식 X 0 X 1 X 2 C X 3 X 4 X 5 CX 6 X 7 X 8 X 9...,0,0,0,0,0,0,0,0,0


In [11]:
df['title'].isnull().sum()
df['summary'].isnull().sum()

np.int64(2)

In [8]:
df['status'].unique()

array(['등록', '공개', '거절', '취하', '포기', '소멸'], dtype=object)

In [10]:
### 새로운 컬럼 ongoing 추가 (출원 특허가 계속 중 혹은 등록 되었는지 여부)
### 무효, 취하, 취소, 포기, 소멸, 거절

invalid_words = ['무효', '거절', '취소', '취하' '포기', '소멸']

# status 컬럼의 각 행을 순회하면서 invalid_words에 포함된 단어가 있는지 여부를 판단하여 onGoing 컬럼을 생성
df['ongoing'] = df['status'].apply(lambda x: 0 if any(word in x for word in invalid_words) else 1)

# 결과 확인
df

,status,title,ap_num,application_date,applicant,representative,final_owner,priority_date,international_filing_date,summary,ongoing
0,등록,자동화 폐기물 수거함의 딥러닝 기반 인공지능 운영 시스템 및 방법(OPERATION...,1020200078616,20200626,(주)에이씨아이케미칼아시아,고영갑,(주)에이씨아이케미칼아시아,NaN,NaN,"본 발명의 실시 예에 따른 시스템은, 자동화 폐기물 수거함의 딥러닝 기반 인공지능 ...",1
1,등록,"인공지능(AI)을 통한 딥러닝훈련모듈과, 순위화프레임워크모듈을 활용하여, 법률전문가...",1020170099431,20170807,강준철,이영규,강소라,NaN,NaN,"본 발명은, 인공지능(AI)에 관한 기술과 온라인 문자열 사전에 관한 기술을 활용하...",1
2,공개,인공지능을 이용한 품질검사 시스템 및 방법(System and method for ...,1020220014151,20220203,모던에이아이비전솔루션 주식회사,김정수,NaN,NaN,NaN,"본 발명은 품질검사를 위한 장치 및 방법에 관한 것으로, 본 발명에 따르면, 종래,...",1
3,공개,파이썬 딥러닝 기반 인공지능(AI) 부동산 컨설팅 매물장(Python deep le...,1020210106993,20210812,조국환,NaN,NaN,NaN,NaN,본 발명은 광범위하게 활용되는 인터넷기반 부동산 정보를 때와 장소를 불문하고 기록하...,1
4,거절,PI 기반 빅데이터 퍼스널 컬러 진단 인공지능 딥러닝 알고리즘 시스템 운영방법(PI...,1020230034713,20230316,주식회사 오콜로르,박상열,NaN,NaN,NaN,본 발명은 PI(Personal Identity) 정보 빅데이터를 구축하는 통하여 ...,0
...,...,...,...,...,...,...,...,...,...,...,...
89905,공개,셀 전압 검증을 통한 배터리 제어(BATTERY CONTROL WITH CELL V...,1020247013462,20220922,볼보 카 코오포레이션,양영준,NaN,20210923.0,20220922.0,"본 개시는 전기 및/또는 하이브리드 전기 차량 구동 기술에 관한 것이다. 특히, 전...",1
89906,공개,인접 평면 교란을 회피하기 위해 문제있는 메모리 평면의 격리(ISOLATING PR...,1020237040022,20220503,"웨스턴 디지털 테크놀로지스, 인코포레이티드",양영준,NaN,20210722.0,20220503.0,프로그램 또는 소거와 같은 다중 평면 동작에서 메모리 셀들의 결함 블록들을 검출하고...,1
89907,공개,비디오 코딩을 위한 신경망의 하이브리드 훈련 방법 및 장치(METHODS AND A...,1020237032052,20220302,베이징 다지아 인터넷 인포메이션 테크놀로지 컴퍼니 리미티드,유미특허법인,NaN,20210303.0,20220302.0,비디오 코딩을 위한 신경망의 하이브리드 훈련 방법 및 장치가 제공된다. 이 방법은 ...,1
89908,공개,항균 펩타이드 및 이의 사용 방법(ANTIMICROBIAL PEPTIDES AND ...,1020237023202,20181205,조에티스 서비시즈 엘엘씨,제일특허법인(유),NaN,20171207.0,20181205.0,일반식 X 0 X 1 X 2 C X 3 X 4 X 5 CX 6 X 7 X 8 X 9...,1


In [ ]:
import pandas as pd

# 예시 데이터프레임 생성
data = {
    'id': [1, 2, 3, 4, 5],
    'applicant': ['현대', '기아', '한국생산기술연구원', '타이완세미컨덕터', '서울대']
}
df = pd.DataFrame(data)

# applicant 컬럼의 값 전처리 (공백 제거, 소문자 변환)
df['applicant_processed'] = df['applicant'].str.replace(' ', '').str.lower()

# 소그룹 및 대그룹 정의
소그룹_매핑 = {
    '현대': ['현대', 'hyn', '에이치디'],
    '기아': ['기아', 'kia'],
    '삼성': ['삼성', 'samsung'],
    '엘지': ['엘지', 'lg'],
    '한화': ['한화', 'hanhwa'],
    '엘에스': ['엘에스', 'ls'],
    '서울바이오시스': ['서울바이오시스'],
    '린나이코리아': ['린나이코리아'],
    '한국생산기술연구소': ['한국생산기술연구원', 'kitech', '한국과학기술연구원', 'kist', '한국기계연구원', 'kimm',
                   '한국에너지기술연구원', 'kier', '한국건설기술연구원', 'kict', '한국해양과학기술원', 'kiost',
                   '한국전자통신 연구원', 'etri', '한국세라믹기술원', 'kicet', '한국지질 자원 연구원', 'kigam'],
}

대그룹_매핑 = {
    '국내대기업': ['현대', '기아', '삼성', '엘지', '한화', '엘에스', '서울바이오시스', '린나이코리아'],
    '국내연구소': ['한국생산기술연구소', 'kitech', '한국과학기술연구소', 'kist', '한국기계연구원', 'kimm',
               '한국에너지기술연구원', 'kier', '한국건설기술연구원', 'kict', '한국해양과학기술원', 'kiost',
               '한국전자통신 연구원', 'etri', '한국세라믹기술원', 'kicet', '한국지질 자원 연구원', 'kigam'],
    '국내대학': ['가톨릭대학교', '서울대학교', '고려대학교', '연세대학교', '한국과학기술원', '포항공과대학교', '한양대학교', 
             '성균관대학교', '경희대학교'],
    '해외출원인': ['타이완세미컨덕터', '캐논', '애플', '화웨이', '인텔', '어플라이드머티어리얼스', '도요타', '한도오따이에네루기',
              '고베세이코쇼', '도쿄엘렉트론', '퀄컴', '마이크론', '도레이', '후지', '도시바']
}

# applicant_processed 컬럼을 이용하여 소그룹 및 대그룹 할당
df['applicant_sgrp'] = df['applicant_processed'].apply(
    lambda x: next((key for key, value in 소그룹_매핑.items() if x in value), 'ETC')
)
df['applicant_lgrp'] = df['applicant_processed'].apply(
    lambda x: next((key for key, value in 대그룹_매핑.items() if x in value), 'ETC')
)

print(df[['applicant', 'applicant_sgrp', 'applicant_lgrp']])
